# MarchingFrontGauges

## Identifying bridges within the potential inundation zone
 
This [Jupyter notebook](http://www.jupyter.org) is based on the GeoClaw [MarchingFront](https://www.clawpack.org/gallery/_static/apps/notebooks/geoclaw/MarchingFront.html) notebook in the [Clawpack Gallery](https://www.clawpack.org/gallery/notebooks.html#notebooks).

The module [clawpack.geoclaw.marching_front.py](https://github.com/clawpack/geoclaw/blob/master/src/python/geoclaw/marching_front.py) defines a function `select_by_flooding` that takes as input an array `Ztopo` containing topography elevations on a rectangular grid and returns an array `pt_chosen` of the same shape with values 1 (if chosen) or 0 (if not chosen).  Other inputs specify the criteria used to choose points, as described below.

The basic idea is that chosen points satisfy certain elevation requirements along with connectivity to the coast.  This was originally developed to identify points in a topography DEM where the topography value $Z$ is below MHW, but that should be initialized as dry land because they are in regions protected by dikes or levies.  

Here it is used to identify bridges that are potentially in the tsunami inundation zone and create a list of synthetic gauges to use in GeoClaw simulations.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os,sys
from numpy import ma # masked arrays
from clawpack.visclaw import colormaps, plottools
from clawpack.geoclaw import topotools, marching_front
from clawpack.amrclaw import region_tools

In [ ]:
import pandas as pd
from scipy import interpolate

In [ ]:
CHT = os.environ['CHT']
sys.path.insert(0,os.path.join(CHT,'common_code'))
from center_points import adjust

In [ ]:
zmin = -60.
zmax = 30.

land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})

sea_cmap = colormaps.make_colormap({ 0.0:[0,0,1], 1.:[.8,.8,1]})

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=(zmin,zmax),
                                     data_break=0.)
                                     
sea_cmap_dry = colormaps.make_colormap({ 0.0:[1.0,0.7,0.7], 1.:[1.0,0.7,0.7]})
cmap_dry, norm_dry = colormaps.add_colormaps((land_cmap, sea_cmap_dry),
                                     data_limits=(zmin,zmax),
                                     data_break=0.)

## Sample topography from a 2 arcsecond DEM


In [ ]:
topo = topotools.Topography('/Users/rjl/topo/topofiles/GHWB_2sec.asc', topo_type=3)

In [ ]:
extent = topo.extent
x1,x2,y1,y2 = extent  # unpack

In [ ]:
figure(figsize=(12,6))
plottools.pcolorcells(topo.X, topo.Y, topo.Z, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(48*pi/180.))

<div id="find-Zlow"></div>

### Finding all points below a given elevation


First we choose all points with elevation below 15 m and that are connected to the coast over this topography extent.

In [ ]:
pts_chosen = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=30., max_iters=None) 

In [ ]:
Zmasked = ma.masked_array(topo.Z, logical_not(pts_chosen))

In [ ]:
figure(figsize=(12,6))
plottools.pcolorcells(topo.X, topo.Y, Zmasked, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(topo.y.mean()*pi/180.))

### Create a buffer zone along shore

To select more points along the shore where the topography is steep, we could have first used `max_iters`.  

To illustrate this, we start again and fist use `max_iters = 5` so that at least 5 grid points are selected near the coast, also setting `Z2 = 1e6` (a huge value) so that the arbitrarily high regions will be included if they are within 5 DEM grid cells of the coast:

In [ ]:
pts_chosen = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=1e6, max_iters=5) 

Plot what we have so far:

In [ ]:
Zmasked = ma.masked_array(topo.Z, logical_not(pts_chosen))

figure(figsize=(12,6))
plottools.pcolorcells(topo.X, topo.Y, Zmasked, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(topo.y.mean()*pi/180.))

Then we augment the points already chosen with any points below 30 m and connected to the coast:

In [ ]:
pts_chosen = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=30., 
                                               prev_pts_chosen=pts_chosen,
                                               max_iters=None) 

In [ ]:
Zmasked = ma.masked_array(topo.Z, logical_not(pts_chosen))

figure(figsize=(12,6))
plottools.pcolorcells(topo.X, topo.Y, Zmasked, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(topo.y.mean()*pi/180.))

In [ ]:
pts_chosen_shallow = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=-15., max_iters=None) 

In [ ]:
Zshallow = ma.masked_array(topo.Z, logical_not(pts_chosen_shallow))

In [ ]:
figure(figsize=(12,6))
plottools.pcolorcells(topo.X, topo.Y, Zshallow, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(topo.y.mean()*pi/180.))

Note that this chooses *all* onshore points in addition to offshore points with elevation greater than -15 m.  

We can take the intersection of this set of points with the onshore points previously chosen to get only the points that lie near the coast:

In [ ]:
pts_chosen_nearshore = logical_and(pts_chosen, pts_chosen_shallow)
Znearshore = ma.masked_array(topo.Z, logical_not(pts_chosen_nearshore))

In [ ]:
figure(figsize=(12,6))
plottools.pcolorcells(topo.X, topo.Y, Znearshore, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(48*pi/180.))

In [ ]:
topo.Z.shape, pts_chosen_nearshore.shape

In [ ]:
topo_fcn = interpolate.RegularGridInterpolator((topo.x, topo.y), topo.Z.T,
                                                bounds_error=False, fill_value=99999)
flag_fcn = interpolate.RegularGridInterpolator((topo.x, topo.y), pts_chosen_nearshore.T,
                                                bounds_error=False, fill_value=0)

In [ ]:
x = -124.2
y = 46.4
xypts = array([x,y])
topo_fcn(xypts), flag_fcn(xypts)

In [ ]:
topo_fcn = interpolate.RegularGridInterpolator((topo.x, topo.y),topo.Z.T,
                                               bounds_error=False, fill_value=99999)
flag_fcn = interpolate.RegularGridInterpolator((topo.x, topo.y),pts_chosen.T,
                                               bounds_error=False, fill_value=0)

## Gauges

In [ ]:
fname = 'GraysHarborPacificCounty.csv'
bridges = pd.read_csv(fname)
bridges.shape

In [ ]:
bridges1 = bridges[logical_and(x1 <= bridges['Longitude'], bridges['Longitude'] <= x2)]

In [ ]:
bridges1 = bridges1[logical_and(y1 <= bridges1['Latitude'], bridges1['Latitude'] <= y2)]

In [ ]:
bridges1.shape

In [ ]:
gaugenos = array(bridges1['ID'])
xg = bridges1['Longitude']
yg = bridges1['Latitude']

In [ ]:
xypts = array([xg,yg]).T
xypts.shape

In [ ]:
topo_xypts = topo_fcn(xypts)
flag_xypts = flag_fcn(xypts)

In [ ]:
# print out all gauges, topo value, and flag:

if 0:
    for k,topoval in enumerate(topo_xypts):
        print('Gauge %s: topo = %5.1fm,   flag = %.2f' \
              % (gaugenos[k],topo_xypts[k],flag_xypts[k]))
    

## make kml file:

Gauges selected are shown as red, others are yellow.

In [ ]:
fname_kml = os.path.splitext(fname)[0] + '_flags.kml'

with open(fname_kml,'w') as kml_file:
    kml_file.write("""<?xml version="1.0" encoding="UTF-8"?>
        <kml xmlns="http://www.opengis.net/kml/2.2"
        xmlns:gx="http://www.google.com/kml/ext/2.2">
        <Document><name>%s</name>

        <Style id="Red">
        <IconStyle><color>FF0000FF</color></IconStyle>
        </Style>

        <Style id="Yellow">
        <IconStyle><color>FF00FFFF</color></IconStyle>
        </Style>
        """  % fname)

    for gaugeno,x,y,topopt,flagpt in zip(gaugenos, xg, yg, topo_xypts, flag_xypts):

        if flagpt > 0:
            kml_file.write("""
                <Placemark><name>%s</name>
                <description>x=%.5f, y=%.5f, topo=%5.1fm, flag=%.2f</description>
                <styleUrl>#%s</styleUrl>
                <Point>
                <coordinates>
                %.9f, %.9f, 0.0
                </coordinates>
                </Point>
                </Placemark>
                """ % (gaugeno,x,y,topopt,flagpt,'Red',x,y))


        else:
            # adjust so x,y are at cell centers of 1/3" grid:
            #x = adjust(x, x_edge, dx, verbose=True)
            #y = adjust(y, y_edge, dy, verbose=True)
            kml_file.write("""
                <Placemark><name>%s</name>
                <description>x=%.5f, y=%.5f, topo=%5.1fm, flag=%.2f</description>
                <styleUrl>#%s</styleUrl>
                <Point>
                <coordinates>
                %.9f, %.9f, 0.0
                </coordinates>
                </Point>
                </Placemark>
                """ % (gaugeno,x,y,topopt,flagpt,'Yellow',x,y))

    kml_file.write("\n</Document>\n</kml>")

print('\nCreated %s' % fname_kml)